<a href="https://colab.research.google.com/github/yashsharan/nnfl_project/blob/master/Clevr_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Run this if you want to acess your files in google d

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import json
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input, Dropout, CuDNNLSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.layers import Embedding

from sklearn.preprocessing import LabelEncoder

In [0]:
#Opening the json file 
file1 = open('/content/drive/My Drive/nnfl_project/Quest_Answers.json', 'r')
json_file = json.load(file1)

In [0]:
quest = json_file['quest_answers']

In [0]:
d = pd.DataFrame(quest)
d.head()
d.shape

(135020, 3)

In [0]:
d['Answer'] = d['Answer'].astype('str')
data = pd.DataFrame()
data['Answer'] = d['Answer'][:135019,]
data['Image'] = d['Image'][:135019,]
data['Question'] = d['Question'][:135019,]

In [0]:
#Using label encoder to encode the labels
labelencoder = LabelEncoder()
labelencoder.fit(data['Answer'])
y = labelencoder.transform(data['Answer'])
num_classes = len(list(labelencoder.classes_))
print(len(y))

135019


In [0]:
print(y[0:100])

[ 0  9 18 15 14 19  9  9 10  9 10  9  9  9  9  9 10  9  9  0  1  2  1  9
  9 10 10  0  0  1  0 20 22 15 23 19 15 18 12 24 23 17 22  2  9 23 14 22
 15  0 23 21 19 15  3  2  1  2  1  1  1  1  4  9 23 12 22 13 10  9  9 10
  9  9  9  9 10  9 10  9  9  9  9  9  0  9 15 22 14 23  1 10 23 13 12 10
  9 10  9 10]


In [0]:
#Mapping all thelabel numbers to thier labels
print(dict(zip(labelencoder.classes_,labelencoder.transform(labelencoder.classes_))))

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, 'False': 9, 'True': 10, 'blue': 11, 'brown': 12, 'cube': 13, 'cyan': 14, 'cylinder': 15, 'gray': 16, 'green': 17, 'large': 18, 'metal': 19, 'purple': 20, 'red': 21, 'rubber': 22, 'small': 23, 'sphere': 24, 'yellow': 25}


In [0]:

#Reading the images from the images folder and resizing them to 64X64 array.
path = "/content/drive/My Drive/nnfl_project/images/"
array_of_images = []
labels = []
dic = {}
from os import listdir
import cv2
size = 64,64
def addImages(images,directory):
    try:
        for i in range(10000):
            progress=(i/10000)*100
            print(progress,end="\r")
            img = cv2.imread(directory + images[i])
            array_of_images.append(img)
            dic[images[i]] = i
    except:
        print(cv2.error)
    
def transform(img_data):
    X = []
    for img in img_data:
        img = cv2.resize(img,size)
        X.append(img)
    return np.array(X)

In [0]:
#Chaning the directory to where the files are present
os.chdir('/content/drive/My Drive/nnfl_project/')

In [0]:
X_img=[]

In [0]:
addImages(sorted(listdir(path)),path)
X_img = transform(array_of_images)    
    

In [0]:
X_img.shape

(10000, 64, 64, 3)

In [0]:
x1=np.array(array_of_images)
np.save('img_arr.npy',x1)

In [0]:
np.save('X_img.npy',X_img)

In [0]:
num_words = 1000
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(data['Question'].values)
X = tokenizer.texts_to_sequences(data['Question'].values)
word_index = tokenizer.word_index

max_length_of_text = 100
X = pad_sequences(X, maxlen=max_length_of_text)

In [0]:
import pickle
file=open("tokenizer.pkl","wb")
pickle.dump(tokenizer,file)
file.close()

In [0]:
#uncomment this if you want to use word2vec for your LSTM model

'''#Creating embeddings
embeddings_index = {}
glove_txt = open('/content/drive/My Drive/nnfl_project/glove.6B.50d.txt')
for line in glove_txt:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
glove_txt.close()

print('Found %s word vectors.' % len(embeddings_index))
max_length_of_text = 100'''

Found 400000 word vectors.


In [0]:
##uncomment this if you want to use word2vec for your LSTM model
'''EMBEDDING_DIM = 50
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector'''

In [0]:
#normalizing the image array
X_norm = np.array(X_img, np.float32) / 255.

In [0]:
mean_img = X_norm.mean(axis=0)

std_dev = X_norm.std(axis = 0)



In [0]:
X_norm_new = (X_norm - mean_img)/ std_dev

In [0]:
np.save('X_norm_new.npy',X_norm_new)

In [0]:
#run this cell if you want to load your save numpy array
X_norm_new=np.load('X_norm_new.npy')

In [0]:
X_norm_new.shape

(10000, 64, 64, 3)

In [0]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_length_of_text,
                            trainable=False)

In [0]:
question_input = Input((max_length_of_text, ))
x2 = embedding_layer(question_input)
x2 = LSTM(128,return_sequences = True)(x2)
x2 = Dropout(0.4)(x2)
x2 = LSTM(128,return_sequences = True)(x2)
x2 = Dropout(0.4)(x2)
x2 = LSTM(128, return_sequences=False)(x2)
x2 = Dropout(0.4)(x2)


In [0]:
from keras.models import Sequential,load_model
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, concatenate,BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [0]:
'''model = Sequential()
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# COMPILE
image_input = Input(shape=(64,64, 3))
encoded_image = model(image_input)'''

In [0]:
#CNN model
model = Sequential()
#model.add(BatchNormalization(input_shape=Xtrain.shape[1:]))
model.add(Conv2D(32, kernel_size=(3, 3), activation= 'relu',padding= 'same'))
model.add(Conv2D(32, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.3))

model.add(Conv2D(64, kernel_size=(3, 3), activation= 'relu', padding='same'))
model.add(Conv2D(64, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation= 'relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))



model.add(Flatten())  
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

image_input = Input(shape=(64,64, 3))
encoded_image = model(image_input)

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 62, 62, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [0]:
#Merging your cnn and lstm model
merged = concatenate([x2, encoded_image])

In [0]:
output = Dense(num_classes, activation='sigmoid')(merged)
clevr_model = Model(inputs=[question_input,image_input], outputs=output)


In [0]:
clevr_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [0]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')
checkpointer = ModelCheckpoint(filepath='weights3.best.eda.hdf5', verbose=1, save_best_only=True)

In [0]:
#conactinating your inage names and Questions
X_merge1 = np.zeros((135019,1))
for i in range(len(X_merge1)):
    try:
      X_merge1[i] = dic[data['Image'][i]+'.jpg']
    except:
      pass

X_final = np.concatenate((X,X_merge1),axis = 1)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_final,y, test_size = 0.2, random_state = 47)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(108015, 101) (108015,)
(27004, 101) (27004,)


In [0]:
X_image_train=X_train[:,100].astype(int)

In [0]:
X_norm_new[X_image_train[0]]

In [0]:
LSTM = X_train[:,0:100]
X_image_test = X_test[:,100].astype(int)

In [0]:
LSTM_test = X_test[:,0:100]
Xvalid=[LSTM_test, X_norm_new[X_image_test] ]

In [0]:
clevr_model.fit(x = [LSTM,X_norm_new[X_image_train]],y = y_train,epochs = 20,validation_data=(Xvalid,y_test), batch_size = 512,callbacks=[checkpointer], verbose=1)

In [0]:
y_pred = clevr_model.predict(x = [LSTM_test, X_norm_new[X_image_test] ], batch_size = 32)

In [0]:
clevr_model.save('clevr3.hdf5')

In [0]:
model = load_model('clevr3.hdf5')

In [0]:
#clevr_model.fit(x = [LSTM,X_img[X_image]],y = y_train,epochs = 30,validation_data=(Xvalid,y_test), batch_size = 256,callbacks=[checkpointer], verbose=1)

In [0]:
len(y_pred)

6000

In [0]:
for i in range(5):
  y_output = y_pred[i]
  for label in y_output.argsort()[::-1][:5]:
      print(str(round(y_output[label]*100,2)).zfill(5), "% ", labelencoder.inverse_transform(label))

In [0]:
#predcting the answers and saving them in csv format
predict_ques = model.predict(x = [LSTM_test, X_norm_new[X_image_test] ], batch_size = 256)





predict_ques = np.argmax(predict_ques, axis = 1)

labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: 'false',
    10: 'true',
    11: 'blue',
    12: 'brown',
    13: 'cube',
    14: 'cyan',
    15: 'cylinder',
    16: 'gray',
    17: 'green',
    18: 'large',
    19: 'metal',
    20: 'purple',
    21: 'red',
    22: 'rubber',
    23: 'small',
    24: 'sphere',
    25: 'yellow'
}

predict_ques = [labels[i] for i in predict_ques]


# In[25]:


new_df = pd.DataFrame({"Index": [i for i in range(len(predict_ques))], "Answer": predict_ques})
# new_df = pd.DataFrame({"Index": indices, "Answer": pred})
new_df.to_csv("solution.csv", index=False)
